# Fitting curves to point clouds using numpy and scipy

<div class="alert alert-block alert-danger">
    <b>Note for contributors:</b> Remember to run <code>Kernel > Restart & Clear output</code> before adding any changes to git!</div>

**In this tutorial, we make our lives a bit easier and use some pre-defined fitting functions.**

In [ ]:
# Numerical operations:
import numpy as np

# Plotting library:
import matplotlib.pyplot as plt

# Minimizing/fitting library:
import scipy.optimize

<div class="alert alert-block alert-info">
<b>Remember:</b> You can always get help about any function, e.g. 
    <code>np.array</code> 
by typing

``np.array?``
</div>

## Fitting lines or polynomial

Fitting a polynomial to a point cloud.
For this we use the ``polyfit`` function of numpy (https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html). 

Source: https://stackoverflow.com/questions/19165259/

Let's start by defining some points and their x and y projections:

In [ ]:
points = np.array([(1, 1), (2, 4), (3, 1), (9, 3), (10, 10)])
x = points[:,0]
y = points[:,1]

Plot them:

<div class="alert alert-block alert-success">
<b>Exercise 1 [super-easy]:</b> Plot the points
</div>

Fitting a polynomial of degree ``deg`` is easy as calling:

In [ ]:
coeffs = np.polyfit(x, y, deg=3)

The vector ``coeffs`` now contains the coefficients of the polynomial as a vector of length ``deg+1``:

In [ ]:
coeffs

It's easy to also get the corresponding function:

In [ ]:
f = np.poly1d(coeffs)

<div class="alert alert-block alert-success">
<b>Exercise 2 [super-easy]:</b> Evaluate the difference $f(x)-y$ and interpret them.
</div>

Let's finally have a look at the fit:

In [ ]:
# Some points where we evaluate our new function
x_new = np.linspace(x[0], x[-1], 50)
y_new = f(x_new)

# Plot the datapoints
plt.plot(x, y, 'ko', label="data")

# Plot our fitted polynomial
plt.plot(x_new, y_new, 'r-', label="fit")

# Add legend etc
plt.legend()
plt.xlim([x[0]-1, x[-1] + 1 ])
plt.show()

<div class="alert alert-block alert-success">
<b>Exercise 3 [easy]:</b> Fit a straight line to the following dataset and determine the line parameters. Bonus points: Also plot the fit as above.
</div>

In [ ]:
x_exercise3 = np.linspace(0, 1, 20)
y_exercise3 = x_exercise3 + np.random.random(len(x_exercise3))

## Fitting Gaussian (or any arbitrary function)

From https://stackoverflow.com/questions/19206332

Again let's generate some points:

In [ ]:
x = np.asarray(range(10))
y = 0.3*x + np.asarray([0,1,2,3,4,5,4,3,2,1])

Here, we define our own function that we want to fit to our data points:

In [ ]:
def gaus(x, norm, mean, sigma):
    # Note that this function takes a whole vector x of data!
    return norm * np.exp(-(x-mean)**2/(2*sigma**2))

This function has 3 free parameters, ``a``, ``x0``, ``sigm``, which we will now fit using ``scipy.optimize.curve_fit`` (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)

In [ ]:
popt, pcov = scipy.optimize.curve_fit(line, x, y)

The variable ``popt`` holds the parameters:

In [ ]:
popt

The other variable, ``pcov`` holds the covariance matrix of the fitted values. 
This gets relevant once you want to give errors on your fitted quantities, but we'll ignore this for now.

In [ ]:
# Plot points
plt.plot(x, y,'ko',label='data')

# Plot our gaussian
# Define some points on the xaxis
x_fine = np.linspace(min(x), max(x), 200)
y_fine = line(x_fine, *popt)
# The *  unpacks the values from popt and uses them as
# parameters.

plt.plot(x_fine, y_fine, 'r-', label='fit')

# ...
plt.legend()
plt.show()

<div class="alert alert-block alert-success">
   <b>Exercise 4a [easy]:</b> 
Modify the above example so that the sigma above is 
    <b>fixed</b> to 1.
</div>

<div class="alert alert-block alert-success">
   <b>Exercise 4b [medium]:</b> 
Let's try to fit the same dataset with a slightly more complicated model that consists of a 
linear part plus a Gaussian, i.e. 
    
```f(x) = x + b + gaus(x, norm, mean, sigma)```

<b>Hint</b>:
Create a new function ``mymodel`` which takes the parameters ``x``, as well as all fitted parameters, i.e. ``b``, ``norm``, ``mean``, ``sigma``. In the definition you can also use the function ``gaus`` from above!
</div>

In [ ]:
def mymodel(x, b, norm, mean, sigma):
    return x + b + gaus(x, norm, mean, sigma)

<div class="alert alert-block alert-success">
   <b>Exercise 4c [hard]:</b> 
Fit the above points as two lines, i.e. 
$$f(x) = \begin{cases}
  a_1 x + b_1 & \text{if}\ x<c\\
  a_2 x + b_2 & \text{if}\ x\geq c\\
\end{cases}$$
    Such that $a_1 c + b_1 = a_2 c + b_2$.
<p>
<b>Hint:</b> 
<ul>
<li>Solve the contraint for, say, <code>b2</code></li>
<li>Define a function <code>line(x, a1, b1, a2, c)</code></li>
<li>Note that <code>x</code> is a vector! Thus <code>if x&lt;c: ...</code> won't work. Rather, take a look at the function <code>np.where</code>.
</ul>
</div>

In [ ]:
def line(x, a1, b1, a2, c):
    return np.where(x<c, a1*x+b1, a2*x+a1*c+b1-a2*c)